In [1]:
import requests
import json
from  time import sleep
import random
from getpass import getpass
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
import time

# Criando o Navegador e Definindo Suas configurações

In [2]:
#detectar a versão do Chrome instalada e baixar o ChromeDriver correspondente
servico = Service(ChromeDriverManager().install())

options = webdriver.ChromeOptions()
# Opções do navegador

options.add_argument("--hide-scrollbars") # esconde as barras de rolagem
options.add_argument("--mute-audio") # desabilita o áudio
options.add_argument("--window-size=800,600") # define o tamanho da janela
options.add_argument("--disable-popup-blocking") # desabilita o bloqueador de pop-ups

options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

# Inicializar o navegador utilizando o undetected-chromedriver com as opções ajustadas
navegador = uc.Chrome(service=servico, options=options)

# Remover o navigator.webdriver
navegador.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
    'source': '''
        Object.defineProperty(navigator, 'webdriver', {
            get: () => undefined
        })
    '''
})

# Espera explícita para garantir que os elementos estejam presentes e clicáveis
wait = WebDriverWait(navegador, 15)
navegador.get("https://lexica.art/?q=anime")


# Loop Principal que realiza a captura das imagens do site

In [3]:

# Lista para armazenar URLs das imagens
img_urls = set()

# Altura inicial da página para comparação
last_height = navegador.execute_script("return document.body.scrollHeight")

scrools = 0

while  scrools < 5:
    # Encontra todas as divisões dentro da divisão principal
    divs = navegador.find_elements(By.XPATH, '/html/body/div/div[2]/div/div[2]/div[6]/div/div')
    
    # Itera sobre cada divisão encontrada
    for div in divs:
        try:
            # Tenta encontrar o elemento de imagem dentro da divisão
            img = div.find_element(By.TAG_NAME, 'img')
            img_url = img.get_attribute('src')
            if img_url:  # Adiciona a URL da imagem se for válida
                img_urls.add(img_url)
        except:
            pass  # Ignora se não encontrar uma imagem na divisão

    # Rola para o fim da página para carregar mais conteúdo
    navegador.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Aguarda o carregamento de novos elementos
    time.sleep(10)

    # Verifica se a altura da página mudou
    new_height = navegador.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  # Sai do loop se não houver mais conteúdo para carregar
    last_height = new_height
    scrools += 1

# Exibe as URLs capturadas
for url in img_urls:
    print(url)


https://image.lexica.art/md2_webp/220919d4-79d5-4c6a-8004-46b4163149b0
https://image.lexica.art/md2_webp/ebd50d3b-298c-4d56-b3d2-5385d4d8cc9c
https://image.lexica.art/md2_webp/05524350-2736-4675-876c-1a29b4f2f7a1
https://image.lexica.art/md2_webp/e84cdf33-6295-41bd-a55f-7e1efdb7343a
https://image.lexica.art/md2_webp/769683af-93b7-4853-b194-5f68c3a27fd0
https://image.lexica.art/md2_webp/0e4bc392-8258-47f5-9678-f67b840bc527
https://image.lexica.art/md2_webp/f04d6395-2521-4d6d-ab19-855eb47ecf13
https://image.lexica.art/md2_webp/c6b145d8-baa6-4741-a757-3356f23eeb42
https://image.lexica.art/md2_webp/20591c4a-4766-479d-a55d-559186e69113
https://image.lexica.art/md2_webp/192f91a1-2ba6-467e-b183-3fa8fe27dd77
https://image.lexica.art/md2_webp/b9a1f590-0c03-4e32-a018-a72247398d6c
https://image.lexica.art/md2_webp/77ceea22-8b23-4be1-9fc3-b8adf41cc951
https://image.lexica.art/md2_webp/9dc89916-0c92-4fad-b429-eb92a9be7718
https://image.lexica.art/md2_webp/31520b47-1c7a-4fff-8317-62e7c66f60cf
https: